In [24]:
import numpy as np
import pandas as pd

In [25]:
df1 = pd.read_csv('R01_300_12_pos_Av_adducts.csv')
df2 = pd.read_csv('R01_300_12_neg_Av_adducts.csv')

In [26]:
df1 = df1.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
df2 = df2.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)

In [27]:
df1['iso_id_esi'] = df1['iso_id'] + '_pos'
df2['iso_id_esi'] = df2['iso_id'] + '_neg'
df = pd.concat([df1, df2], axis=0, sort=False)

In [28]:
df3 = df.set_index('iso_id_esi')
dfloc1 = df3.loc[:, 'M331':'845C']
dfloc2 = df3.loc[:, 'chem_id_s2':'ACN_Adduct']

In [29]:
dfloc1T = dfloc1.T
dfloc1T.columns.name = None

In [30]:
df_cm = dfloc1T.corr()
df_cm = df_cm.where(pd.np.triu(pd.np.ones(df_cm.shape), k=1).astype(bool)).stack().reset_index() #extract upper triangle of the matrix
df_cm = df_cm.dropna()
df_cm = df_cm.rename(index = str, columns = {'level_0':'rows', 'level_1':'columns', 0:'r-coefficient'})
df_cm['id'] = df_cm['rows'] + '+' + df_cm['columns']

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  


In [31]:
# Exract r-coefficients > 0.9
df_cm = df_cm.loc[df_cm['r-coefficient'] >= 0.5]

In [32]:
df_cm.shape

(865, 4)

In [33]:
df_cm

,rows,columns,r-coefficient,id
712,C10H12O_1.0_pos,C10H13N_1.0_pos,0.612362,C10H12O_1.0_pos+C10H13N_1.0_pos
1465,C10H12O4_1.0_pos,C12H14O_1.0_pos,0.665942,C10H12O4_1.0_pos+C12H14O_1.0_pos
1474,C10H12O4_1.0_pos,C12H16O3_3.0_pos,0.728914,C10H12O4_1.0_pos+C12H16O3_3.0_pos
1490,C10H12O4_1.0_pos,C13H16O3_3.0_pos,0.689905,C10H12O4_1.0_pos+C13H16O3_3.0_pos
1620,C10H12O4_1.0_pos,C6H13NO2_1.0_pos,0.525447,C10H12O4_1.0_pos+C6H13NO2_1.0_pos
...,...,...,...,...
253013,C9H17NO2_1.0_neg,C9H17NO3_3.0_neg,0.682408,C9H17NO2_1.0_neg+C9H17NO3_3.0_neg
253014,C9H17NO2_1.0_neg,C9H18O2_1.0_neg,0.817816,C9H17NO2_1.0_neg+C9H18O2_1.0_neg
253027,C9H17NO3_1.0_neg,C9H18O2_1.0_neg,0.558134,C9H17NO3_1.0_neg+C9H18O2_1.0_neg
253050,C9H17NO3_3.0_neg,C9H18O2_1.0_neg,0.758876,C9H17NO3_3.0_neg+C9H18O2_1.0_neg


In [34]:
df_cm['pn_id'] = np.where((df_cm['id'].str.contains('pos') & df_cm['id'].str.contains('neg')), 1, 0)

In [35]:
df_cm['pn_id'].sum()

185

In [36]:
df_cm['Formula_rows'] = df_cm['rows'].str.split('_').apply(lambda x: x[0])
df_cm['Formula_rows'] = df_cm['Formula_rows'].str.replace(' ', '')
df_cm['Formula_columns'] = df_cm['columns'].str.split('_').apply(lambda x: x[0])
df_cm['Formula_columns'] = df_cm['Formula_columns'].str.replace(' ', '')

In [37]:
df_cm

,rows,columns,r-coefficient,id,pn_id,Formula_rows,Formula_columns
712,C10H12O_1.0_pos,C10H13N_1.0_pos,0.612362,C10H12O_1.0_pos+C10H13N_1.0_pos,0,C10H12O,C10H13N
1465,C10H12O4_1.0_pos,C12H14O_1.0_pos,0.665942,C10H12O4_1.0_pos+C12H14O_1.0_pos,0,C10H12O4,C12H14O
1474,C10H12O4_1.0_pos,C12H16O3_3.0_pos,0.728914,C10H12O4_1.0_pos+C12H16O3_3.0_pos,0,C10H12O4,C12H16O3
1490,C10H12O4_1.0_pos,C13H16O3_3.0_pos,0.689905,C10H12O4_1.0_pos+C13H16O3_3.0_pos,0,C10H12O4,C13H16O3
1620,C10H12O4_1.0_pos,C6H13NO2_1.0_pos,0.525447,C10H12O4_1.0_pos+C6H13NO2_1.0_pos,0,C10H12O4,C6H13NO2
...,...,...,...,...,...,...,...
253013,C9H17NO2_1.0_neg,C9H17NO3_3.0_neg,0.682408,C9H17NO2_1.0_neg+C9H17NO3_3.0_neg,0,C9H17NO2,C9H17NO3
253014,C9H17NO2_1.0_neg,C9H18O2_1.0_neg,0.817816,C9H17NO2_1.0_neg+C9H18O2_1.0_neg,0,C9H17NO2,C9H18O2
253027,C9H17NO3_1.0_neg,C9H18O2_1.0_neg,0.558134,C9H17NO3_1.0_neg+C9H18O2_1.0_neg,0,C9H17NO3,C9H18O2
253050,C9H17NO3_3.0_neg,C9H18O2_1.0_neg,0.758876,C9H17NO3_3.0_neg+C9H18O2_1.0_neg,0,C9H17NO3,C9H18O2


In [38]:
df_cm['same_formula'] = np.where(df_cm['Formula_rows'] == df_cm['Formula_columns'], 1, 0)

In [39]:
df_cm['same_formula'].sum()

35

In [40]:
df_dup = df_cm[(df_cm['same_formula'] == 1) & (df_cm['pn_id'] == 1)]
df_dup.head()

,rows,columns,r-coefficient,id,pn_id,Formula_rows,Formula_columns,same_formula
4553,C10H14O_3.0_pos,C10H14O_5.0_neg,0.514498,C10H14O_3.0_pos+C10H14O_5.0_neg,1,C10H14O,C10H14O,1
20517,C11H11NO2_1.0_pos,C11H11NO2_1.0_neg,0.743048,C11H11NO2_1.0_pos+C11H11NO2_1.0_neg,1,C11H11NO2,C11H11NO2,1
21879,C11H11NO2_3.0_pos,C11H11NO2_2.0_neg,0.510063,C11H11NO2_3.0_pos+C11H11NO2_2.0_neg,1,C11H11NO2,C11H11NO2,1
70124,C14H26O4_1.0_pos,C14H26O4_1.0_neg,0.651808,C14H26O4_1.0_pos+C14H26O4_1.0_neg,1,C14H26O4,C14H26O4,1
89598,C18H18O4S_1.0_pos,C18H18O4S_1.0_neg,0.536135,C18H18O4S_1.0_pos+C18H18O4S_1.0_neg,1,C18H18O4S,C18H18O4S,1


In [41]:
df.head()

,chem_id_s1,M331,M332,M317,M524,M102,M341,M330,M345,M340,...,Mass_av,Retention Time_av,Sodium_Adduct,Potasium_Adduct,Ammonium_Adduct,ACN_Adduct,iso_id_esi,Formate_Adduct,H2O_Adduct,CO2_Adduct
0,158.0848_1.8326_75,23014.583414,35781.743242,630102.449070,11495.685179,297645.722596,114589.220618,1.526543e+06,101597.021085,236089.213100,...,158.08480,1.839971,0.0,0.0,0.0,0.0,C10H10N2_1.0_pos,NaN,NaN,NaN
1,148.0886_3.6386_30,10460.647408,22674.974161,22478.453200,10830.129767,46811.341571,27959.377011,1.155142e+04,5633.801102,26549.384877,...,148.08865,3.697920,0.0,0.0,0.0,0.0,C10H12O_1.0_pos,NaN,NaN,NaN
2,196.0739_11.7317_477,11323.048129,11816.145003,11821.121662,12678.024245,6034.328408,18383.357132,1.089179e+04,10890.162708,10792.346778,...,196.07410,11.970576,0.0,0.0,0.0,0.0,C10H12O4_1.0_pos,NaN,NaN,NaN
3,147.1045_3.2268_364,57081.423799,54767.399215,49932.178513,38288.600153,3325.918526,72691.996597,4.950163e+04,79429.757331,7759.361744,...,147.10455,3.320171,0.0,0.0,0.0,0.0,C10H13N_1.0_pos,NaN,NaN,NaN
4,147.1047_7.2359_363,5268.617733,3069.552750,11034.774489,4001.319531,3115.572284,3158.377996,1.068680e+03,1742.412358,3674.003545,...,147.10475,7.379868,0.0,0.0,0.0,0.0,C10H13N_2.0_pos,NaN,NaN,NaN


In [42]:
df['pos_neg_dup'] = np.where(df['iso_id_esi'].isin(df_dup['rows']), 1, 0)
df['neg_pos_dup'] = np.where(df['iso_id_esi'].isin(df_dup['columns']), 1, 0)

In [43]:
df['neg_pos_dup'].sum()

10

In [44]:
df = df.fillna(0)

In [45]:
df

,chem_id_s1,M331,M332,M317,M524,M102,M341,M330,M345,M340,...,Sodium_Adduct,Potasium_Adduct,Ammonium_Adduct,ACN_Adduct,iso_id_esi,Formate_Adduct,H2O_Adduct,CO2_Adduct,pos_neg_dup,neg_pos_dup
0,158.0848_1.8326_75,23014.583414,35781.743242,630102.449070,11495.685179,297645.722596,114589.220618,1.526543e+06,101597.021085,236089.213100,...,0.0,0.0,0.0,0.0,C10H10N2_1.0_pos,0.0,0.0,0.0,0,0
1,148.0886_3.6386_30,10460.647408,22674.974161,22478.453200,10830.129767,46811.341571,27959.377011,1.155142e+04,5633.801102,26549.384877,...,0.0,0.0,0.0,0.0,C10H12O_1.0_pos,0.0,0.0,0.0,0,0
2,196.0739_11.7317_477,11323.048129,11816.145003,11821.121662,12678.024245,6034.328408,18383.357132,1.089179e+04,10890.162708,10792.346778,...,0.0,0.0,0.0,0.0,C10H12O4_1.0_pos,0.0,0.0,0.0,0,0
3,147.1045_3.2268_364,57081.423799,54767.399215,49932.178513,38288.600153,3325.918526,72691.996597,4.950163e+04,79429.757331,7759.361744,...,0.0,0.0,0.0,0.0,C10H13N_1.0_pos,0.0,0.0,0.0,0,0
4,147.1047_7.2359_363,5268.617733,3069.552750,11034.774489,4001.319531,3115.572284,3158.377996,1.068680e+03,1742.412358,3674.003545,...,0.0,0.0,0.0,0.0,C10H13N_2.0_pos,0.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,160.0639_5.7146_33,7539.435471,104818.089726,4011.933121,13480.667976,113395.643158,60719.560627,5.238938e+03,45589.479347,117230.137490,...,0.0,0.0,0.0,0.0,C9H8N2O_1.0_neg,0.0,0.0,0.0,0,0
408,132.0576_5.3568_283,635.582616,1768.427586,1406.367253,7809.952567,13035.288043,1433.358954,1.827622e+03,1669.775180,4544.108838,...,0.0,0.0,0.0,0.0,C9H8O_1.0_neg,0.0,0.0,0.0,0,0
409,148.0526_2.1169_256,358159.014834,157068.385277,701612.501761,372464.792116,244608.739591,384360.588364,2.211682e+05,376849.914047,447320.709265,...,0.0,0.0,0.0,0.0,C9H8O2_1.0_neg,0.0,0.0,0.0,0,0
410,224.0686_1.4658_199,1411.812994,3944.466896,2420.638777,6484.454022,7854.335337,2513.399086,4.025823e+03,2361.064485,2980.946775,...,0.0,0.0,0.0,0.0,C9H8O3_1.0_neg,0.0,0.0,0.0,0,0


In [46]:
df.to_csv('R01_combined_posnegBaC.csv')